In [26]:
import os
import cv2 as cv
import numpy as np

In [27]:
yolo_path='C:/Users/Amir/Downloads/Video/maryam sadeghi/deep learning/ipynb/yolo3 opencv'
coco_path='C:/Users/Amir/Downloads/Video/maryam sadeghi/deep learning/ipynb/yolo3 opencv/coco.names'

In [28]:
classes=[]
with open(coco_path,'r') as f:
        classes = [line.strip() for line in f.readlines()]
  


In [29]:
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

## 80 different classes


In [30]:
len(classes)

80

In [31]:
yolo_net = cv.dnn.readNet(os.path.join(yolo_path,"yolov3.weights"), os.path.join(yolo_path,"yolov3.cfg"))
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]

In [48]:
image=cv.imread('04_tehran.png')
height, width, channel =image.shape

In [49]:
img_blob= cv.dnn.blobFromImage(image, 0.00392, (416,416), (0,0,0) , True, crop=False)

In [50]:
yolo_net.setInput(img_blob)
outputs=yolo_net.forward(output_layers)

In [51]:
outputs[0].shape       # 85 = heigh width cordination points + 80 classes

(507, 85)

In [52]:
class_ids=[]
confidences=[]
boxes=[]
for out in outputs:
    for detection in out:
        scores=detection[5:]
        class_id=np.argmax(scores)
        confidence=scores[class_id]
        if confidence>0.5:
            center_x= int(detection[0]*width)
            center_y= int(detection[1]*height)
            w= int(detection[2]*width)
            h= int(detection[3]*height)
            # rectangle coordinates 
            x=int(center_x- w/2)
            y=int(center_y- h/2)
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)


In [53]:
colors=np.random.uniform(0, 255, size=(len(classes), 3))
indexes= cv.dnn.NMSBoxes(boxes,confidences, 0.5, 0.4)      #non maximum suppression  to keep only the maximum box
for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h= boxes[i]
        label=str(classes[class_ids[i]])
        color= colors[class_ids[i]]
        cv.rectangle(image, (x,y), (x+w,y+h),color,2)
        cv.putText(image, label, (x,y+30), cv.FONT_HERSHEY_PLAIN, 1, color, 1)
cv.imwrite('04_tehran_det.png', image)

True

## object detection in video

In [55]:
cap = cv.VideoCapture("driving_camera.mp4")
video_cod = cv.VideoWriter_fourcc(*'XVID')
video_output = cv.VideoWriter('driving_camera_det_yolov3_cv.avi',
                      video_cod,
                      10,
                      (1280,720))
while(cap.isOpened()):
    ret, img = cap.read()
    if ret == True:
        #img = cv.resize(frame, None, fx=0.8, fy=0.8)
        height, width, channels = img.shape
        #print(height, width)
        blob = cv.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        yolo_net.setInput(blob)
        outs = yolo_net.forward(output_layers)
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        font = cv.FONT_HERSHEY_PLAIN
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = colors[class_ids[i]]
                cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv.putText(img, label, (x, y + 30), font, 1, color, 1)
        cv.imshow('Frame',img)
        video_output.write(img)

        if cv.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
video_output.release()
cv.destroyAllWindows()

## WEBCAM

In [56]:
cap = cv.VideoCapture(0)

while(cap.isOpened()):
    ret, img = cap.read()
    if ret == True:
        #img = cv.resize(frame, None, fx=0.8, fy=0.8)
        height, width, channels = img.shape
        print(height, width)
        blob = cv.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        yolo_net.setInput(blob)
        outs = yolo_net.forward(output_layers)
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        print(indexes)
        font = cv.FONT_HERSHEY_PLAIN
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = colors[class_ids[i]]
                cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv.putText(img, label, (x, y + 30), font, 2, color, 2)
        cv.imshow('Frame',img)
        if cv.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
    
cap.release()
cv.destroyAllWindows()

480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
[0]
480 640
()
480 640
[0]
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
[0]
480 640
()
480 640
()
480 640
[1 3]
480 640
()
480 640
[1]
480 640
()
480 640
[1]
480 640
[0]
480 640
[0]
480 640
[1]
480 640
[1]
480 640
[1]
480 640
[1]
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
[0]
480 640
[0]
480 640
()
480 640
()
480 640
()
480 640
()
480 640
[0]
480 640
()
480 640
[1]
480 640
()
480 640
()
480 640
[1]
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
480 640
()
48